# Optuna: Hyperparameter Tuning with LGBM

In [ ]:
#!pip install optuna 
import optuna

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('../input/30days-folds/train_folds.csv')
test  = pd.read_csv('../input/30-days-of-ml/test.csv')
sub = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

In [ ]:
cat_cols = ['cat'+str(i) for i in range(10)]
con_cols = ['cont'+str(i) for i in range(14)]

## Encode categorical features

In [ ]:
imp_cols = cat_cols+con_cols
data=train[imp_cols]
target=train['target']

In [ ]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    ordinal_encoder = OrdinalEncoder()
    train_x[cat_cols] = ordinal_encoder.fit_transform(train_x[cat_cols])
    test_x[cat_cols] = ordinal_encoder.transform(test_x[cat_cols])
    
    std_scaler = StandardScaler()
    train_x[con_cols] = std_scaler.fit_transform(train_x[con_cols])
    test_x[con_cols] = std_scaler.transform(test_x[con_cols])
    
    param = {
        'metric': 'rmse', 
        'random_state': 42,
        'n_estimators': 20000,
        'device_type':'gpu',
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = LGBMRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

* Objective of our fuction is to minimize the RMSE hence direction='minimize', n_trials = number of executions

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

# Quick Visualization for Hyperparameter Optimization Analysis

In [ ]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
'''plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
went and which parts of the space were explored more.'''
optuna.visualization.plot_slice(study)

In [ ]:
#plot_contour: plots parameter interactions on an interactive chart. You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['num_leaves',
                            'max_depth',
                            'subsample',
                            'learning_rate',
                            'subsample'])

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

# LGBMRegressor model with the best hyperparameters

In [ ]:
params=study.best_params   
params['random_state'] = 42
params['n_estimators'] = 20000 
params['metric'] = 'rmse'
params['device_type'] = 'gpu'

In [ ]:
# change the param name, differnt in optuna and lgbm
params['cat_smooth'] = params.pop('min_data_per_groups')

In [ ]:
params

In [ ]:
preds = np.zeros(test.shape[0])
preds_all = np.zeros((test.shape[0], 5))
rmse=[]  # list contains rmse for each fold

for fold in range(5):
    X_tr =  train[train.kfold != fold].reset_index(drop=True)
    X_val = train[train.kfold == fold].reset_index(drop=True)
    y_tr = X_tr.target
    y_val = X_val.target 
    xtest = test.copy()
    
    X_tr = X_tr[imp_cols]
    X_val = X_val[imp_cols]
    xtest = xtest[imp_cols]
    
    ordinal_encoder = OrdinalEncoder()
    X_tr[cat_cols] = ordinal_encoder.fit_transform(X_tr[cat_cols])
    X_val[cat_cols] = ordinal_encoder.transform(X_val[cat_cols])
    xtest[cat_cols] = ordinal_encoder.transform(xtest[cat_cols])
    
    std_scaler = StandardScaler()
    X_tr[con_cols] = std_scaler.fit_transform(X_tr[con_cols])
    X_val[con_cols] = std_scaler.transform(X_val[con_cols])
    xtest[con_cols] = std_scaler.transform(xtest[con_cols])
    
    model = LGBMRegressor(**params)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds = model.predict(xtest)
    
    preds_all[:, fold] = preds

    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(fold+1,rmse[fold])


In [ ]:
from optuna.integration import lightgbm as lgb
lgb.plot_importance(model, max_num_features=10, figsize=(10,10))
plt.show()


# Submission

In [ ]:
np.savetxt("all_data.csv", preds_all, delimiter=",")

sub['target']=np.mean(preds_all, axis = 1)
sub.to_csv('submission.csv', index=False)